In [1]:
pip install python-geohash



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from ipyleaflet import Map, Marker, Circle, WidgetControl, Popup,AwesomeIcon
from ipywidgets import IntSlider, Layout, HTML
from geopy.distance import geodesic
import geohash as gh
import pandas as pd

pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('res.csv',on_bad_lines='skip', header = 0)
df

,listing_id,price,sentiment_score,latitude,longitude,name,listing_url,review_scores_rating,squared_error
0,Amsterdam1000033344100120272,299.0,0.974100,52.367080,4.863996,Garden retreat in the city,https://www.airbnb.com/rooms/1000033344100120272,5.00,0.000671
1,Amsterdam1000066837570466403,200.0,0.126000,52.418358,4.902166,Het Zaanse sprookjeshuis,https://www.airbnb.com/rooms/1000066837570466403,5.00,0.763876
2,Amsterdam1000621220103148407,371.0,0.623667,52.368499,4.890162,Dam Center Oasis,https://www.airbnb.com/rooms/1000621220103148407,4.67,0.096307
3,Amsterdam1001038303846389335,178.0,0.297450,52.349950,5.016227,* * * Stylish Artsy 2-bedroom Beach Suite- i(h...,https://www.airbnb.com/rooms/1001038303846389335,4.75,0.425821
4,Amsterdam1001235717002673120,229.0,0.000000,52.352900,4.920050,Apartment Amsterdam East,https://www.airbnb.com/rooms/1001235717002673120,5.00,1.000000
...,...,...,...,...,...,...,...,...,...
101667,paris999894571235431060,275.0,0.699900,48.858087,2.308663,Townhouse in Paris · ★New · 2 bedrooms · 2 bed...,https://www.airbnb.com/rooms/999894571235431060,5.00,0.090060
101668,paris999910359815339007,84.0,0.629133,48.849700,2.294160,Rental unit in Paris · ★4.67 · 1 bedroom · 1 b...,https://www.airbnb.com/rooms/999910359815339007,4.67,0.092944
101669,paris999929481353480141,127.0,0.648533,48.837636,2.395612,Rental unit in Paris · ★5.0 · 1 bedroom · 1 be...,https://www.airbnb.com/rooms/999929481353480141,5.00,0.123529
101670,paris999949819215823638,226.0,0.000000,48.836420,2.281120,Rental unit in Paris · ★New · 1 bedroom · 1 be...,https://www.airbnb.com/rooms/999949819215823638,5.00,1.000000


In [4]:
# Reading london travel score data
df_travel = pd.read_csv('travel_scored_airbnb.csv', header=0)
df_travel

,Unnamed: 0,listing_id,travel_score
0,0,Rome1000038128653158461,5.0
1,1,Rome1000085573090116607,4.0
2,2,Rome1000087947380625661,1.0
3,3,Rome1000161932420441438,2.0
4,4,Rome1000512975918982495,4.0
...,...,...,...
77517,77517,paris999894571235431060,5.0
77518,77518,paris999910359815339007,3.0
77519,77519,paris999929481353480141,2.0
77520,77520,paris999949819215823638,3.0


In [5]:
df = df.merge(df_travel, left_on = 'listing_id', right_on = 'listing_id')
df

,listing_id,price,sentiment_score,latitude,longitude,name,listing_url,review_scores_rating,squared_error,Unnamed: 0,travel_score
0,Rome1000038128653158461,45.0,0.163900,41.898351,12.503785,Bed and breakfast in Rome · ★4.29 · 1 bedroom ...,https://www.airbnb.com/rooms/1000038128653158461,4.29,0.481775,0,5.0
1,Rome1000085573090116607,51.0,0.369957,41.897855,12.503528,Bed and breakfast in Rome · ★4.86 · 1 bedroom ...,https://www.airbnb.com/rooms/1000085573090116607,4.86,0.362456,1,4.0
2,Rome1000087947380625661,42.0,-0.006571,41.916294,12.421239,Bed and breakfast in Rome · ★4.71 · 1 bedroom ...,https://www.airbnb.com/rooms/1000087947380625661,4.71,0.899788,2,1.0
3,Rome1000161932420441438,42.0,0.248200,41.917597,12.515703,Condo in Rome · ★5.0 · 1 bedroom · 1 bed · 1 p...,https://www.airbnb.com/rooms/1000161932420441438,5.00,0.565203,3,2.0
4,Rome1000512975918982495,119.0,0.212300,41.909511,12.491012,Condo in Rome · ★5.0 · Studio · 2 beds · 1 bath,https://www.airbnb.com/rooms/1000512975918982495,5.00,0.620471,4,4.0
...,...,...,...,...,...,...,...,...,...,...,...
77516,paris999894571235431060,275.0,0.699900,48.858087,2.308663,Townhouse in Paris · ★New · 2 bedrooms · 2 bed...,https://www.airbnb.com/rooms/999894571235431060,5.00,0.090060,77517,5.0
77517,paris999910359815339007,84.0,0.629133,48.849700,2.294160,Rental unit in Paris · ★4.67 · 1 bedroom · 1 b...,https://www.airbnb.com/rooms/999910359815339007,4.67,0.092944,77518,3.0
77518,paris999929481353480141,127.0,0.648533,48.837636,2.395612,Rental unit in Paris · ★5.0 · 1 bedroom · 1 be...,https://www.airbnb.com/rooms/999929481353480141,5.00,0.123529,77519,2.0
77519,paris999949819215823638,226.0,0.000000,48.836420,2.281120,Rental unit in Paris · ★New · 1 bedroom · 1 be...,https://www.airbnb.com/rooms/999949819215823638,5.00,1.000000,77520,3.0


In [6]:
# Return the length of prefix size of geohash based on distance
def get_length(distance):
    if distance <= 0.019:
        return 8
    elif distance <= 0.076:
        return 7
    elif distance <= 0.61:
        return 6
    elif distance <= 2.4:
        return 5
    elif distance <= 20:
        return 4
    elif distance <= 73:
        return 3
    elif distance <= 630:
        return 2
    else:
        return 1

# Binary search to find the start and end of data within distance of pinned location 
def find_start_end_prefix(data, distance, pinned):
    pinned_geohash = gh.encode(pinned[0], pinned[1])
    prefix = pinned_geohash[:(get_length(distance))]

    geo_list = data['geohash'].values.tolist()
    
    l = 0
    r = len(geo_list)-1
    # Find the first location with matching prefix to prefix
    while l<r:
        mid = int((l+r)/2)
        if geo_list[mid][:(len(prefix))] >= prefix:
            r = mid
        else:
            l = mid+1
    
    start = l

    l = 0
    r = len(geo_list)-1
    # Find the last location with matching prefix to prefix
    while (r-l)>1:
        mid = int((l+r)/2)
        if geo_list[mid][:(len(prefix))] <= prefix:
            l = mid
        else:
            r = mid-1
    
    end = r

    return data.iloc[start:(end+1),:]


In [7]:
# Get geohash code columns
df['latitude'] = df['latitude'].apply(lambda x: float(x))
df['longitude'] = df['longitude'].apply(lambda x: float(x))
df['geohash'] = df.apply(lambda x: gh.encode(x['latitude'],x['longitude']), axis=1)
df.sort_values('geohash', ascending=True, inplace=True)

df

,listing_id,price,sentiment_score,latitude,longitude,name,listing_url,review_scores_rating,squared_error,Unnamed: 0,travel_score,geohash
13912,Toronto41660993,95.0,0.911014,43.591110,-79.544760,"☆Private 1 Bdrm Apt☆ 15mins to Downtown, w/par...",https://www.airbnb.com/rooms/41660993,4.95,0.006239,13913,1.0,dpxrvzt28ezg
16041,Toronto791172921733137748,67.0,0.800640,43.591669,-79.543356,Women Only - 2nd Floor Modern Apt @ Long Branc...,https://www.airbnb.com/rooms/791172921733137748,5.00,0.039744,16042,1.0,dpxrvzw7ckp1
13767,Toronto38783172,95.0,0.819280,43.592213,-79.542175,"Beautiful, Modern Apt by Lake Ontario and Nature",https://www.airbnb.com/rooms/38783172,4.85,0.022717,13768,1.0,dpxrvzxphcrm
13713,Toronto38026639,115.0,0.858556,43.592213,-79.542175,"Beautiful, Sun-filled Apt by Lake Ontario & Na...",https://www.airbnb.com/rooms/38026639,4.89,0.014267,13714,1.0,dpxrvzxphcrm
11914,Toronto1004250252777319277,65.0,0.978900,43.591752,-79.541022,3753 Lake Shore (11),https://www.airbnb.com/rooms/1004250252777319277,5.00,0.000445,11914,1.0,dpxrvzxurgdg
...,...,...,...,...,...,...,...,...,...,...,...,...
22456,london19587434,90.0,0.707656,51.608160,0.225970,Home in Greater London · ★4.56 · 2 bedrooms · ...,https://www.airbnb.com/rooms/19587434,4.56,0.041757,22457,1.0,u10jt4ux36r3
27482,london32308720,80.0,0.936700,51.612160,0.221320,Rental unit in Greater London · 1 bedroom · 1 ...,https://www.airbnb.com/rooms/32308720,5.00,0.004007,27483,1.0,u10jt59nt5un
46759,london915107545774964827,42.0,0.773164,51.606353,0.241000,Home in Greater London · ★4.79 · 1 bedroom · 1...,https://www.airbnb.com/rooms/915107545774964827,4.79,0.034164,46760,1.0,u10jt6xkz9w8
46333,london907288884018021957,40.0,0.652278,51.606600,0.241089,Home in Greater London · ★4.44 · 1 bedroom · 1...,https://www.airbnb.com/rooms/907288884018021957,4.44,0.055565,46334,1.0,u10jt6xw1z8b


# Visualization

In [8]:
# Create an AwesomeIcon
icon = AwesomeIcon(
    name='map-pin',  # FontAwesome icon name
    marker_color='red',  # Marker color
    icon_color='white',  # Icon color inside the marker
    spin=False  # Whether the icon should spin
)

# Initialize map example for london 
m = Map(center=(51.53501,0.02568), zoom=10) #map(center=(latitude,longitude), zoom = 10), we can change another latitude and longtitude for different city 


# Create a marker and a circle for the center for london 
marker = Marker(location=(51.53501,0.02568), draggable=True, icon=icon) #marker = Marker(location=(latitude,longitude)) 
m.add_layer(marker)
circle = Circle(location=(51.53501,0.02568), radius=5000, color="red", fill_color="red")#circle = Circle(location=(latitude,longitude)) 
m.add_layer(circle)

# Define the radius update function
def update_radius(change):
    circle.radius = change['new'] * 1000
    update_map()

radius_slider = IntSlider(description='Radius (km):', min=1, max=30, value=1)
radius_slider.observe(update_radius, names='value')

# Widget to control the radius slider
slider_control = WidgetControl(widget=radius_slider, position='topright')
m.add_control(slider_control)

# Function to update map based on selected location and radius
def update_map(event=None):
    global marker, circle
    circle.location = marker.location  # Update circle location to marker's position
    

    # Calculate and filter locations within the radius
    center_point = marker.location
    radius_in_km = radius_slider.value


    # Clear existing markers (except the main marker)
    for layer in list(m.layers)[3:]:
        if isinstance(layer, Marker):
            m.remove_layer(layer)
    
    cnt = 0
    result_sub = find_start_end_prefix(df, radius_in_km, center_point)
    result_sub.sort_values('sentiment_score',ascending=False,inplace=True)
    for _, row in result_sub.iterrows():
        if cnt>=10:
            break
        loc_coords = gh.decode(row['geohash'])
        if geodesic(center_point, loc_coords).km <= radius_in_km:
            new_marker = Marker(location=loc_coords, draggable=False)
            m.add_layer(new_marker)
            # Create a Popup with HTML content
            popup_html = HTML()
            popup_html.value = f"<b>{row['name']}</b><br><a href='{row['listing_url']}' target='_blank'>Listing URL</a><br>Price: ${row['price']}</a><br>Score: {row['sentiment_score']}</a><br>Travel score: {row['travel_score']}"
            popup = Popup(child=popup_html, close_button=False, auto_close=False, close_on_escape_key=False, auto_pan=False)
            new_marker.popup = popup

            cnt+=1

# Handle marker drag to update map
marker.observe(update_map, 'location')

# Display the map
m


Map(center=[51.53501, 0.02568], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

/var/folders/r1/6jgmv3zj23n01bnfwt4_379c0000gn/T/ipykernel_68888/3530160277.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_sub.sort_values('sentiment_score',ascending=False,inplace=True)
/var/folders/r1/6jgmv3zj23n01bnfwt4_379c0000gn/T/ipykernel_68888/3530160277.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_sub.sort_values('sentiment_score',ascending=False,inplace=True)
/var/folders/r1/6jgmv3zj23n01bnfwt4_379c0000gn/T/ipykernel_68888/3530160277.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://panda